# Pandas教程 | 数据处理三板斧——map、apply、applymap详解

文字来源：https://zhuanlan.zhihu.com/p/100064394

In [4]:
import pandas as pd
import numpy as np

In [11]:
boolean = [ True, False ]
gender = [ "男", "女" ]
color = ["white", "black", "yellow"]
df = pd.DataFrame({
    "height": np.random.randint(150,190,10),
    "weight": np.random.randint(40,90,10),
    "smoker": [boolean[x] for x in np.random.randint(0,2,10)],
    "gender": [gender[x] for x in np.random.randint(0,2,10)],
    "age": np.random.randint(15,90,10),
    "color": [color[x] for x in np.random.randint(0,len(color),10) ]
  }
)
df

,height,weight,smoker,gender,age,color
0,163,84,False,男,85,black
1,183,48,False,男,25,yellow
2,186,43,False,女,89,white
3,185,45,False,女,76,yellow
4,167,71,True,女,85,yellow
5,155,72,False,男,87,black
6,167,80,True,男,37,yellow
7,159,53,False,男,53,white
8,181,89,True,女,62,yellow
9,175,47,False,女,57,yellow


### Series数据处理
#### 1. map用法
如果需要把数据集中gender列的男替换为1，女替换为0，怎么做呢？绝对不是用for循环实现，使用Series.map()可以很容易做到，最少仅需一行代码。

可以使用字典进行映射

In [9]:
df['gender'] = df['gender'].map({"男": 1, "女": 0})
df

,height,weight,smoker,gender,age,color
0,157,73,True,0,40,yellow
1,176,63,True,1,83,white
2,150,82,True,0,35,yellow
3,164,44,True,0,32,black
4,152,60,True,0,65,yellow
5,164,70,False,0,73,black
6,150,40,True,1,55,white
7,183,85,True,0,78,black
8,182,50,True,1,19,white
9,167,75,False,1,63,white


也可以使用函数。

In [12]:
df["gender"] = df["gender"].map(lambda x: 1 if x =="男" else 0)
df

,height,weight,smoker,gender,age,color
0,163,84,False,1,85,black
1,183,48,False,1,25,yellow
2,186,43,False,0,89,white
3,185,45,False,0,76,yellow
4,167,71,True,0,85,yellow
5,155,72,False,1,87,black
6,167,80,True,1,37,yellow
7,159,53,False,1,53,white
8,181,89,True,0,62,yellow
9,175,47,False,0,57,yellow


那map在实际过程中是怎么运行的呢？请看下面的图解:

![](https://pic1.zhimg.com/80/v2-2a08d3b4ae37e7847d1e9d9a92375904_720w.jpg)

![](https://pic3.zhimg.com/80/v2-29a652a10f80c1b137fa703c7fcf912a_720w.jpg)

不论是利用字典还是函数进行映射，map方法都是把对应的数据逐个当作参数传入到字典或函数中，得到映射后的值。

## 2. apply

假设在数据统计的过程中，年龄age列有较大误差，需要对其进行调整（加上或减去一个值），由于这个加上或减去的值未知，故在定义函数时，需要加多一个参数bias，此时用map方法是操作不了的（传入map的函数只能接收一个参数），apply方法则可以解决这个问题。

In [19]:
def apply_age(x,bias):
  return x + bias

#以元组的方式传入额外的参数
df["age"] = df["age"].apply(apply_age,args=(-3,))
df

,height,weight,smoker,gender,age,color
0,163,84,False,1,70,black
1,183,48,False,1,10,yellow
2,186,43,False,0,74,white
3,185,45,False,0,61,yellow
4,167,71,True,0,70,yellow
5,155,72,False,1,72,black
6,167,80,True,1,22,yellow
7,159,53,False,1,38,white
8,181,89,True,0,47,yellow
9,175,47,False,0,42,yellow


总而言之，对于Series而言，map可以解决绝大多数的数据处理需求，但如果需要使用较为复杂的函数，则需要用到apply方法。



### DataFrame数据处理
#### 1. apply
对DataFrame而言，apply是非常重要的数据处理方法，它可以接收各种各样的函数（Python内置的或自定义的），处理方式很灵活，下面通过几个例子来看看apply的具体使用及其原理。

在进行具体介绍之前，首先需要介绍一下DataFrame中axis的概念，在DataFrame对象的大多数方法中，都会有axis这个参数，它控制了你指定的操作是沿着0轴还是1轴进行。axis=0代表操作对列columns进行，axis=1代表操作对行row进行，如下图所示。


![](https://pic4.zhimg.com/80/v2-c3ef5114a1eb9240dfc4920e632f035b_720w.jpg)


如果还不是很了解，没关系，下面会分别对apply沿着0轴以及1轴的操作进行讲解，继续往下走。

假设现在需要对data中的数值列分别进行求和的操作，这时可以用apply进行相应的操作，因为是对列进行操作，所以需要指定axis=0，使用下面的两行代码可以很轻松地解决我们的问题。



In [20]:

df[["height","weight","age"]].apply(np.sum)

height    1721
weight     632
age        506
dtype: int64

In [21]:
# 沿着0轴求和

df[["height","weight","age"]].apply(np.sum, axis=0) 

height    1721
weight     632
age        506
dtype: int64

![](https://pic3.zhimg.com/80/v2-3e7aa714ac4478c6b4b7da0b4dd3746e_720w.jpg)

当沿着轴0（axis=0）进行操作时，会将各列(columns)默认以Series的形式作为参数，传入到你指定的操作函数中，操作后合并并返回相应的结果。

那如果在实际使用中需要按行进行操作（axis=1）,那整个过程又是怎么实现的呢？

在数据集中，有身高和体重的数据，所以根据这个，我们可以计算每个人的BMI指数（体检时常用的指标，衡量人体肥胖程度和是否健康的重要标准），计算公式是：体重指数BMI=体重/身高的平方（国际单位kg/㎡），因为需要对每个样本进行操作，这里使用axis=1的apply进行操作，代码如下：

In [23]:
def BMI(series):
    weight = series["weight"]
    height = series["height"]/100
    BMI = weight/height**2
    return BMI

df["BMI"] = df.apply(BMI,axis=1)
df

,height,weight,smoker,gender,age,color,BMI
0,163,84,False,1,70,black,31.615793
1,183,48,False,1,10,yellow,14.333065
2,186,43,False,0,74,white,12.429183
3,185,45,False,0,61,yellow,13.148283
4,167,71,True,0,70,yellow,25.458066
5,155,72,False,1,72,black,29.968783
6,167,80,True,1,22,yellow,28.685145
7,159,53,False,1,38,white,20.964361
8,181,89,True,0,47,yellow,27.166448
9,175,47,False,0,42,yellow,15.346939


![](https://pic4.zhimg.com/80/v2-98f8b09e26abe17e850ed125950fecdf_720w.jpg)

当 `apply` 设置了 `axis=1` 对行进行操作时，会默认将每一行数据以Series的形式（Series的索引为列名）传入指定函数，返回相应的结果。

总结一下对 `DataFrame` 的 `apply` 操作：

1. 当axis=0时，对每列columns执行指定函数；当axis=1时，对每行row执行指定函数。
2. 无论axis=0还是axis=1，其传入指定函数的默认形式均为Series，可以通过设置 `raw=True` 传入 `numpy` 数组。
3. 对每个Series执行结果后，会将结果整合在一起返回（若想有返回值，定义函数时需要return相应的值）
4. 当然，DataFrame的apply和Series的apply一样，也能接收更复杂的函数，如传入参数等，实现原理是一样的，具体用法详见官方文档。

### 2. applymap
applymap的用法比较简单，会对DataFrame中的每个单元格执行指定函数的操作，虽然用途不如apply广泛，但在某些场合下还是比较有用的，如下面这个例子。

为了演示的方便，新生成一个DataFrame

In [24]:
df = pd.DataFrame(
    {
        "A":np.random.randn(5),
        "B":np.random.randn(5),
        "C":np.random.randn(5),
        "D":np.random.randn(5),
        "E":np.random.randn(5),
    }
)
df

,A,B,C,D,E
0,0.892988,-1.532808,-0.303719,0.368746,-0.722312
1,0.031863,-0.549665,0.627299,-1.564693,0.483534
2,0.279619,0.586686,0.737646,-1.512648,-1.009894
3,-0.646960,0.260647,0.654822,-1.921295,1.401469
4,-0.331035,0.798392,0.708446,-1.598650,-1.076704


现在想将DataFrame中所有的值保留两位小数显示，使用applymap可以很快达到你想要的目的，代码和图解如下：

In [25]:
df.applymap(lambda x: "%.2f" % x)

,A,B,C,D,E
0,0.89,-1.53,-0.30,0.37,-0.72
1,0.03,-0.55,0.63,-1.56,0.48
2,0.28,0.59,0.74,-1.51,-1.01
3,-0.65,0.26,0.65,-1.92,1.40
4,-0.33,0.80,0.71,-1.60,-1.08


![](https://pic1.zhimg.com/80/v2-d90bd5f9a7bdbbf811063df36c9b3720_720w.jpg)